In [1]:
from sqlalchemy import create_engine, text

In [2]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [3]:
engine = connection_yandex_cloud_demo(echo=True)

In [4]:
SCHEMA_NAME = 'public'

### Оптимизация с помощью индекса

**Создадим таблицу для экспериментов и заполним её данными**

In [5]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.ticket_flights_copy;

CREATE TABLE {SCHEMA_NAME}.ticket_flights_copy
 (LIKE bookings.ticket_flights);
 
INSERT INTO {SCHEMA_NAME}.ticket_flights_copy
SELECT * FROM bookings.ticket_flights
'''

engine.execute(sql)

/var/folders/g7/ydw5y9ls4_l8cbw38l0v1p_80000gn/T/ipykernel_15123/2364493803.py:11: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(sql)


2024-03-13 20:26:22,022 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-03-13 20:26:22,024 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:26:22,048 INFO sqlalchemy.engine.Engine select current_schema()
2024-03-13 20:26:22,049 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:26:22,072 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-03-13 20:26:22,073 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:26:22,098 INFO sqlalchemy.engine.Engine 
DROP TABLE IF EXISTS public.ticket_flights_copy;

CREATE TABLE public.ticket_flights_copy
 (LIKE bookings.ticket_flights);
 
INSERT INTO public.ticket_flights_copy
SELECT * FROM bookings.ticket_flights

2024-03-13 20:26:22,099 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:29:01,642 INFO sqlalchemy.engine.Engine COMMIT


**Посмотрим кол-во записей в созданной таблице**

In [6]:
sql = f'''
 SELECT count(*)
   FROM {SCHEMA_NAME}.ticket_flights_copy
'''

engine.execute(sql).fetchall()[0][0]

2024-03-13 20:30:20,973 INFO sqlalchemy.engine.Engine 
 SELECT count(*)
   FROM public.ticket_flights_copy

2024-03-13 20:30:20,976 INFO sqlalchemy.engine.Engine [raw sql] {}


8391852

**Сделаем запрос на поиск данных по перелету с идентификатором 130525**

In [7]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

2024-03-13 20:31:01,455 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525

2024-03-13 20:31:01,459 INFO sqlalchemy.engine.Engine [generated in 0.00301s] {}


[('Gather  (cost=1000.00..114647.56 rows=70 width=32) (actual time=138.461..784.592 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..113640.56 rows=29 width=32) (actual time=123.343..756.183 rows=31 loops=3)',),
 ('        Filter: (flight_id = 130525)',),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 1.978 ms',),
 ('Execution Time: 784.665 ms',)]

**Попробуем использовать нечеткий поиск, посмотрим как изменлось время выполнения запроса**

In [8]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

2024-03-13 20:31:46,847 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%%130525%%'

2024-03-13 20:31:46,849 INFO sqlalchemy.engine.Engine [generated in 0.00191s] {}


[('Gather  (cost=1000.00..132392.09 rows=2685 width=32) (actual time=242.552..1556.815 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..131123.59 rows=1119 width=32) (actual time=230.331..1535.403 rows=31 loops=3)',),
 ("        Filter: ((flight_id)::text ~~ '%130525%'::text)",),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 0.060 ms',),
 ('Execution Time: 1556.864 ms',)]

**Создадим индекс на поле bookings.ticket_flights_copy.flight_id**

In [9]:
sql = f'''
DROP INDEX IF EXISTS ticket_flights_copy_flight_id_index;

CREATE INDEX ticket_flights_copy_flight_id_index 
          ON {SCHEMA_NAME}.ticket_flights_copy (flight_id);
'''

engine.execute(sql)

2024-03-13 20:36:03,361 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS ticket_flights_copy_flight_id_index;

CREATE INDEX ticket_flights_copy_flight_id_index 
          ON public.ticket_flights_copy (flight_id);

2024-03-13 20:36:03,363 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:36:29,222 INFO sqlalchemy.engine.Engine COMMIT


**Выполним запросы после создания индекса, сравним время выполнения с предыдущими результатом**

In [10]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

2024-03-13 20:36:52,528 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525

2024-03-13 20:36:52,530 INFO sqlalchemy.engine.Engine [cached since 351.1s ago] {}


[('Index Scan using ticket_flights_copy_flight_id_index on ticket_flights_copy tfc  (cost=0.43..69.50 rows=70 width=32) (actual time=0.064..0.170 rows=93 loops=1)',),
 ('  Index Cond: (flight_id = 130525)',),
 ('Planning Time: 0.148 ms',),
 ('Execution Time: 0.198 ms',)]

In [11]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

2024-03-13 20:36:55,682 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%%130525%%'

2024-03-13 20:36:55,683 INFO sqlalchemy.engine.Engine [cached since 308.9s ago] {}


[('Gather  (cost=1000.00..132392.09 rows=2685 width=32) (actual time=216.360..1764.875 rows=93 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on ticket_flights_copy tfc  (cost=0.00..131123.59 rows=1119 width=32) (actual time=205.084..1740.930 rows=31 loops=3)',),
 ("        Filter: ((flight_id)::text ~~ '%130525%'::text)",),
 ('        Rows Removed by Filter: 2797253',),
 ('Planning Time: 0.053 ms',),
 ('Execution Time: 1764.927 ms',)]

In [ ]:
784.665 ms
1556.864 ms

**Создадим копию таблицы flights и выполним запрос с фильтрацией по двум атрибутам**

In [12]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.flights_copy;

CREATE TABLE {SCHEMA_NAME}.flights_copy
 (LIKE bookings.flights);
 
INSERT INTO {SCHEMA_NAME}.flights_copy
SELECT * FROM bookings.flights
'''

engine.execute(sql)

2024-03-13 20:39:29,720 INFO sqlalchemy.engine.Engine 
DROP TABLE IF EXISTS public.flights_copy;

CREATE TABLE public.flights_copy
 (LIKE bookings.flights);
 
INSERT INTO public.flights_copy
SELECT * FROM bookings.flights

2024-03-13 20:39:29,723 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:39:33,426 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-03-13 20:40:19,492 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-03-13 20:40:19,494 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Gather  (cost=1000.00..5639.69 rows=1198 width=63) (actual time=0.229..36.407 rows=1584 loops=1)',),
 ('  Workers Planned: 1',),
 ('  Workers Launched: 1',),
 ('  ->  Parallel Seq Scan on flights_copy fc  (cost=0.00..4519.89 rows=705 width=63) (actual time=8.128..24.490 rows=792 loops=2)',),
 ("        Filter: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('        Rows Removed by Filter: 106642',),
 ('Planning Time: 0.290 ms',),
 ('Execution Time: 36.516 ms',)]

**Создадим составной индекс**

In [14]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_airports_index;

CREATE INDEX flights_copy_airports_index 
          ON {SCHEMA_NAME}.flights_copy (departure_airport, arrival_airport);
'''

engine.execute(sql)

2024-03-13 20:40:38,943 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS flights_copy_airports_index;

CREATE INDEX flights_copy_airports_index 
          ON public.flights_copy (departure_airport, arrival_airport);

2024-03-13 20:40:38,945 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:40:39,344 INFO sqlalchemy.engine.Engine COMMIT


**Выполним этот запрос повторно**

In [21]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE 1=1
    AND fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-03-13 20:43:23,075 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE 1=1
  AND fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-03-13 20:43:23,078 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_airports_index on flights_copy fc  (cost=0.29..995.47 rows=1198 width=63) (actual time=0.020..0.218 rows=1584 loops=1)',),
 ("  Index Cond: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('Planning Time: 0.073 ms',),
 ('Execution Time: 0.292 ms',)]

In [ ]:
'Execution Time: 36.516 ms'

**Выведем только те поля, которые участвуют в индексе**

In [22]:
sql = f'''
EXPLAIN ANALYZE
 SELECT fc.departure_airport
       ,fc.arrival_airport
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

2024-03-13 20:43:50,190 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT fc.departure_airport
       ,fc.arrival_airport
   FROM public.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'

2024-03-13 20:43:50,192 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Only Scan using flights_copy_airports_index on flights_copy fc  (cost=0.29..26.26 rows=1198 width=8) (actual time=0.027..0.175 rows=1584 loops=1)',),
 ("  Index Cond: ((departure_airport = 'DME'::bpchar) AND (arrival_airport = 'LED'::bpchar))",),
 ('  Heap Fetches: 0',),
 ('Planning Time: 0.066 ms',),
 ('Execution Time: 0.254 ms',)]

**Допустим, нам необходимо посмотреть данные за определённую дату о совершенных полётах**

In [23]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) = '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-03-13 20:44:59,692 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE DATE(fc.actual_departure) = '2017-09-06'

2024-03-13 20:44:59,696 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Gather  (cost=1000.00..5627.29 rows=1074 width=63) (actual time=44.261..48.175 rows=0 loops=1)',),
 ('  Workers Planned: 1',),
 ('  Workers Launched: 1',),
 ('  ->  Parallel Seq Scan on flights_copy fc  (cost=0.00..4519.89 rows=632 width=63) (actual time=41.910..41.911 rows=0 loops=2)',),
 ("        Filter: (date(actual_departure) = '2017-09-06'::date)",),
 ('        Rows Removed by Filter: 107434',),
 ('Planning Time: 0.068 ms',),
 ('Execution Time: 48.205 ms',)]

**Сравним время выполнения фильтрации по равенству и наравенству**

In [24]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) > '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-03-13 20:45:16,112 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE DATE(fc.actual_departure) > '2017-09-06'

2024-03-13 20:45:16,114 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Seq Scan on flights_copy fc  (cost=0.00..5847.00 rows=71622 width=63) (actual time=38.666..38.667 rows=0 loops=1)',),
 ("  Filter: (date(actual_departure) > '2017-09-06'::date)",),
 ('  Rows Removed by Filter: 214867',),
 ('Planning Time: 0.050 ms',),
 ('Execution Time: 38.699 ms',)]

**Создадим индекс на колонку с датой отправления**

In [25]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_actual_departure_index;

CREATE INDEX flights_copy_actual_departure_index ON {SCHEMA_NAME}.flights_copy (actual_departure);
'''

engine.execute(sql)

2024-03-13 20:46:53,851 INFO sqlalchemy.engine.Engine 
DROP INDEX IF EXISTS flights_copy_actual_departure_index;

CREATE INDEX flights_copy_actual_departure_index ON public.flights_copy (actual_departure);

2024-03-13 20:46:53,853 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:46:54,210 INFO sqlalchemy.engine.Engine COMMIT


**Проверим, ускорился ли наш запрос**

In [26]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure = '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-03-13 20:47:15,296 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.actual_departure = '2017-09-06'

2024-03-13 20:47:15,299 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_actual_departure_index on flights_copy fc  (cost=0.42..2.44 rows=1 width=63) (actual time=0.055..0.056 rows=0 loops=1)',),
 ("  Index Cond: (actual_departure = '2017-09-06 00:00:00+03'::timestamp with time zone)",),
 ('Planning Time: 0.197 ms',),
 ('Execution Time: 0.083 ms',)]

**Сравним время выполнения фильтрации по равенству и наравенству на индексе**

In [27]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure > '2017-09-06'
'''

engine.execute(sql).fetchall()

2024-03-13 20:47:39,158 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.actual_departure > '2017-09-06'

2024-03-13 20:47:39,160 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Index Scan using flights_copy_actual_departure_index on flights_copy fc  (cost=0.42..2.44 rows=1 width=63) (actual time=0.004..0.004 rows=0 loops=1)',),
 ("  Index Cond: (actual_departure > '2017-09-06 00:00:00+03'::timestamp with time zone)",),
 ('Planning Time: 0.132 ms',),
 ('Execution Time: 0.025 ms',)]

**UNION vs UNION ALL**

In [28]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

2024-03-13 20:48:47,704 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM public.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'

2024-03-13 20:48:47,705 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Unique  (cost=43364.47..49273.31 rows=214867 width=170) (actual time=93.103..163.906 rows=214867 loops=1)',),
 ('  ->  Sort  (cost=43364.47..43901.63 rows=214867 width=170) (actual time=93.101..118.825 rows=214867 loops=1)',),
 ('        Sort Key: fc.flight_id, fc.flight_no, fc.scheduled_departure, fc.scheduled_arrival, fc.departure_airport, fc.arrival_airport, fc.status, fc.aircraft_code, fc.actual_departure, fc.actual_arrival',),
 ('        Sort Method: external merge  Disk: 17112kB',),
 ('        ->  Append  (cost=0.00..13842.68 rows=214867 width=170) (actual time=0.019..55.734 rows=214867 loops=1)',),
 ('              ->  Seq Scan on flights_copy fc  (cost=0.00..5309.84 rows=75337 width=63) (actual time=0.019..20.232 rows=75402 loops=1)',),
 ("                    Filter: (scheduled_departure < '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('                    Rows Removed by Filter: 139465',),
 ('              ->  Seq Scan on flights_copy fc2  (cost=0.00..5309.84 row

In [29]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

2024-03-13 20:52:09,571 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYZE 
 SELECT * 
   FROM public.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM public.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'

2024-03-13 20:52:09,575 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Append  (cost=0.00..13842.68 rows=214867 width=63) (actual time=0.020..53.080 rows=214867 loops=1)',),
 ('  ->  Seq Scan on flights_copy fc  (cost=0.00..5309.84 rows=75337 width=63) (actual time=0.020..19.189 rows=75402 loops=1)',),
 ("        Filter: (scheduled_departure < '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('        Rows Removed by Filter: 139465',),
 ('  ->  Seq Scan on flights_copy fc2  (cost=0.00..5309.84 rows=139530 width=63) (actual time=0.008..20.579 rows=139465 loops=1)',),
 ("        Filter: (scheduled_departure >= '2017-01-01 00:00:00+03'::timestamp with time zone)",),
 ('        Rows Removed by Filter: 75402',),
 ('Planning Time: 0.132 ms',),
 ('Execution Time: 60.111 ms',)]

**Временные таблицы**

In [30]:
sql = f'''
EXPLAIN ANALYSE
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
FROM {SCHEMA_NAME}.flights_copy fc
JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id 
'''

engine.execute(sql).fetchall()

2024-03-13 20:53:46,511 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYSE
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
FROM public.flights_copy fc
JOIN public.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id 

2024-03-13 20:53:46,514 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Merge Join  (cost=28421.09..379038.23 rows=8391852 width=91) (actual time=51.199..9177.462 rows=8391852 loops=1)',),
 ('  Merge Cond: (tfc.flight_id = fc.flight_id)',),
 ('  ->  Index Scan using ticket_flights_copy_flight_id_index on ticket_flights_copy tfc  (cost=0.43..203223.22 rows=8391852 width=32) (actual time=0.006..5844.498 rows=8391852 loops=1)',),
 ('  ->  Materialize  (cost=28420.46..29494.79 rows=214867 width=63) (actual time=51.184..560.271 rows=8456116 loops=1)',),
 ('        ->  Sort  (cost=28420.46..28957.62 rows=214867 width=63) (actual time=51.181..91.853 rows=214867 loops=1)',),
 ('              Sort Key: fc.flight_id',),
 ('              Sort Method: external merge  Disk: 17136kB',),
 ('              ->  Seq Scan on flights_copy fc  (cost=0.00..4772.67 rows=214867 width=63) (actual time=0.005..14.432 rows=214867 loops=1)',),
 ('Planning Time: 0.376 ms',),
 ('Execution Time: 9555.550 ms',)]

**Создаём временную таблицу**

In [31]:
sql = f'''
CREATE TEMPORARY TABLE ticket_flights_materialized_{SCHEMA_NAME} AS 
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
  FROM {SCHEMA_NAME}.flights_copy fc
  JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id;
'''

engine.execute(sql)

2024-03-13 20:57:48,491 INFO sqlalchemy.engine.Engine 
CREATE TEMPORARY TABLE ticket_flights_materialized_public AS 
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
  FROM public.flights_copy fc
  JOIN public.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id;

2024-03-13 20:57:48,494 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-13 20:59:04,012 INFO sqlalchemy.engine.Engine COMMIT


**Сделаем запрос к веременной таблице. Как изменилось время выполнения?**

In [32]:
sql = f'''
EXPLAIN ANALYSE
 SELECT tfm.*
   FROM ticket_flights_materialized_{SCHEMA_NAME} tfm;
'''

engine.execute(sql).fetchall()

2024-03-13 20:59:17,076 INFO sqlalchemy.engine.Engine 
EXPLAIN ANALYSE
 SELECT tfm.*
   FROM ticket_flights_materialized_public tfm;

2024-03-13 20:59:17,078 INFO sqlalchemy.engine.Engine [raw sql] {}


[('Seq Scan on ticket_flights_materialized_public tfm  (cost=0.00..172613.70 rows=3561870 width=280) (actual time=0.046..135610.413 rows=8391852 loops=1)',),
 ('Planning Time: 0.104 ms',),
 ('Execution Time: 135951.634 ms',)]

In [ ]:
'Execution Time: 9555.550 ms'